In [18]:
import pandas as pd

In [19]:
df = pd.read_csv("full_sync_summary_0.1keys_100B.csv")

In [20]:
df

,test_type,rdb_threads,num_keys_millions,value_size_bytes,primary_port,replica_port,status,sync_duration_seconds,primary_cpu_time_seconds,primary_io_write_mb_s,replica_cpu_time_seconds,replica_io_read_mb_s
0,full_sync,1,0.1,100,7000,7001,ok,23.641480,2.61,0.000347,12.71,0.0
1,full_sync,2,0.1,100,7000,7001,ok,19.633916,1.64,0.000209,11.55,0.0
2,full_sync,3,0.1,100,7000,7001,ok,18.632451,1.57,0.000220,11.45,0.0
3,full_sync,4,0.1,100,7000,7001,ok,17.630142,1.63,0.000232,11.41,0.0
4,full_sync,6,0.1,100,7000,7001,ok,17.730293,1.38,0.000000,11.29,0.0
5,full_sync,8,0.1,100,7000,7001,ok,17.730071,1.32,0.000231,11.31,0.0
6,full_sync,10,0.1,100,7000,7001,ok,17.732376,1.43,0.000231,11.28,0.0


In [21]:
df = pd.read_csv("full_sync_heavy-json-2mill.csv")

In [22]:
df

,test_type,rdb_threads,num_keys_millions,value_size_bytes,primary_port,status,bgsave_duration_seconds,primary_cpu_time_seconds,primary_net_write_mb_s
0,primary_bgsave,1,1.0,100,7000,ok,103.805421,103.47,89.273862
1,primary_bgsave,2,1.0,100,7000,ok,48.299751,93.70,191.277986
2,primary_bgsave,3,1.0,100,7000,ok,36.037740,102.15,255.984164
3,primary_bgsave,4,1.0,100,7000,ok,26.307607,97.54,350.494213
4,primary_bgsave,6,1.0,100,7000,ok,18.575728,100.72,495.465530
5,primary_bgsave,8,1.0,100,7000,ok,13.956728,97.36,658.772083
6,primary_bgsave,10,1.0,100,7000,ok,13.265815,97.42,694.656984
7,primary_bgsave,15,1.0,100,7000,ok,13.272289,104.66,695.847400
8,primary_bgsave,20,1.0,100,7000,ok,13.167936,105.44,695.188360
9,primary_bgsave,25,1.0,100,7000,ok,13.157423,106.35,696.638677


In [23]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from typing import List
import io

def plot_rdb_throughput(df: pd.DataFrame, title: str):
    """
    Generates an interactive Plotly graph showing network throughput vs.
    number of threads, with an ideal throughput line for comparison.

    Args:
        df (pd.DataFrame): A DataFrame containing 'rdb_threads' and
                          'primary_net_write_mb_s' columns.
        title (str): The main title for the graph.
    """
    if not isinstance(df, pd.DataFrame) or df.empty:
        print("Error: Input 'df' must be a non-empty DataFrame.")
        return

    # Get all unique thread counts for x-axis ticks
    all_num_threads = sorted(df['rdb_threads'].unique())

    fig = go.Figure()

    # Plot the actual throughput data
    fig.add_trace(
        go.Scatter(
            x=df['rdb_threads'],
            y=df['primary_net_write_mb_s'],
            mode='lines+markers',
            name='Actual Throughput',
            marker=dict(symbol='circle', size=8),
            line=dict(width=2, color=px.colors.qualitative.Plotly[0]),
            showlegend=True
        )
    )

    # Add Ideal Network Throughput line (from iPerf3 test)
    # The value 3237.50 MB/s is from our iPerf3 test results.
    ideal_throughput_mb_s = 3237.50 
    fig.add_trace(
        go.Scatter(
            x=all_num_threads,
            y=[ideal_throughput_mb_s] * len(all_num_threads),
            mode='lines',
            name='Ideal Throughput (Network Limit)',
            line=dict(dash='dash', color='gray'),
            showlegend=True
        )
    )

    # Update layout for a beautiful graph
    fig.update_layout(
        title_text=f'{title} - Throughput vs. Number of Threads',
        height=600, width=800,
        showlegend=True,
        hovermode="x unified",
        template="plotly_white",
        title_x=0.5,
        xaxis=dict(
            title_text='Number of Threads',
            tickmode='array',
            tickvals=all_num_threads
        ),
        yaxis=dict(
            title_text='Network Throughput (MB/s)',
            range=[0, ideal_throughput_mb_s * 1.1]
        )
    )

    fig.show()
    fig.write_json("rdb_throughput_plot_go.json")

# User-provided data as a string
user_data_string = """	test_type	rdb_threads	num_keys_millions	value_size_bytes	primary_port	status	bgsave_duration_seconds	primary_cpu_time_seconds	primary_net_write_mb_s
0	primary_bgsave	1	1.0	100	7000	ok	103.805421	103.47	89.273862
1	primary_bgsave	2	1.0	100	7000	ok	48.299751	93.70	191.277986
2	primary_bgsave	3	1.0	100	7000	ok	36.037740	102.15	255.984164
3	primary_bgsave	4	1.0	100	7000	ok	26.307607	97.54	350.494213
4	primary_bgsave	6	1.0	100	7000	ok	18.575728	100.72	495.465530
5	primary_bgsave	8	1.0	100	7000	ok	13.956728	97.36	658.772083
6	primary_bgsave	10	1.0	100	7000	ok	13.265815	97.42	694.656984
7	primary_bgsave	15	1.0	100	7000	ok	13.272289	104.66	695.847400
8	primary_bgsave	20	1.0	100	7000	ok	13.167936	105.44	695.188360
9	primary_bgsave	25	1.0	100	7000	ok	13.157423	106.35	696.638677
10	primary_bgsave	30	1.0	100	7000	ok	13.062020	106.20	703.329440
"""
# Convert the string to a DataFrame
df_user = pd.read_csv(io.StringIO(user_data_string), sep='\t')

# Call the new plotting function
plot_rdb_throughput(df_user, title='RDB Transfer')

In [24]:
plot_rdb_throughput(df, "title")

In [25]:
df_replica = pd.read_csv("full_sync_heavy-json-2mill-replica.csv")

In [26]:
df_replica

,test_type,rdb_threads,num_keys_millions,value_size_bytes,sync_duration_seconds,primary_ip,replica_ip,replica_port,replica_cpu_time_seconds,replica_net_read_mb_s,status
0,replica_full_sync,1,10.0,100,23.530062,10.128.0.8,10.128.0.9,7001,7.97,51.283130,success
1,replica_full_sync,2,10.0,100,17.621643,10.128.0.8,10.128.0.9,7001,7.44,68.287366,success
2,replica_full_sync,3,10.0,100,15.520386,10.128.0.8,10.128.0.9,7001,7.36,77.536062,success
3,replica_full_sync,4,10.0,100,15.421679,10.128.0.8,10.128.0.9,7001,7.47,78.033571,success
4,replica_full_sync,6,10.0,100,14.518671,10.128.0.8,10.128.0.9,7001,7.56,82.881191,success
5,replica_full_sync,8,10.0,100,14.218560,10.128.0.8,10.128.0.9,7001,8.32,84.633827,success
6,replica_full_sync,10,10.0,100,14.722507,10.128.0.8,10.128.0.9,7001,7.51,81.715881,success
7,replica_full_sync,15,10.0,100,14.823518,10.128.0.8,10.128.0.9,7001,7.64,81.130260,success
8,replica_full_sync,20,10.0,100,13.922445,10.128.0.8,10.128.0.9,7001,7.71,86.386649,success
9,replica_full_sync,25,10.0,100,13.721187,10.128.0.8,10.128.0.9,7001,7.53,87.642494,success
